In [1]:
import io
import os
import sys
import string
import numpy as np
import pandas as pd
from tensorflow import keras
from __future__ import print_function
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import LambdaCallback, ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import Dense, Dropout, Activation, LSTM, Bidirectional, Embedding
translator = str.maketrans('', '', string.punctuation)

In [10]:
df=pd.read_csv(r'C:\Users\amjat\Desktop\Large Language models\22 APP\lyrics geneartor\musicoset_songfeatures\lyrics.csv',sep="\t")
df.head()

,song_id,lyrics
0,3e9HZxeyfWwjeyPAMmWSSQ,['[Verse 1]\nThought I\'d end up with Sean\nBu...
1,5p7ujcrUXASCNwRaWNHR1C,"[""[Verse 1]\nFound you when your heart was bro..."
2,2xLMifQCjDGFmkHkpNLD9h,"['[Part I]\n\n[Intro: Drake]\nAstro, yeah\nSun..."
3,3KkXRkHbMCARz0aVfEt68P,NaN
4,1rqqCSm0Qe4I9rUvWncaom,"[""[Intro]\nHigh, high hopes\n\n[Chorus]\nHad t..."


In [24]:
df.dropna(inplace=True)

In [25]:
import re
def clean(df):
    lyrics_row = df['lyrics']
    cleaned_lyrics = re.sub(r'\[.*?\]|\n|\\\'', '', lyrics_row)
    return cleaned_lyrics

In [26]:
df['single_text']=df.apply(clean,axis=1)

In [27]:
df.head()

,song_id,lyrics,single_text
0,3e9HZxeyfWwjeyPAMmWSSQ,['[Verse 1]\nThought I\'d end up with Sean\nBu...,\nThought Id end up with Sean\nBut he wasnt a ...
1,5p7ujcrUXASCNwRaWNHR1C,"[""[Verse 1]\nFound you when your heart was bro...",\nFound you when your heart was broke\nI fille...
2,2xLMifQCjDGFmkHkpNLD9h,"['[Part I]\n\n[Intro: Drake]\nAstro, yeah\nSun...","\n\n\nAstro, yeah\nSun is down, freezin cold\n..."
4,1rqqCSm0Qe4I9rUvWncaom,"[""[Intro]\nHigh, high hopes\n\n[Chorus]\nHad t...","\nHigh, high hopes\n\n\nHad to have high, high..."
5,0bYg9bo50gSsH3LtXe2SQn,"[""[Intro]\nI-I-I don't want a lot for Christma...",\nI-I-I don't want a lot for Christmas\nThere ...


In [28]:
pdf = pd.read_csv(r'C:\Users\amjat\Desktop\Large Language models\22 APP\lyrics geneartor\PoetryFoundationData.csv',quotechar='"')
pdf.head()

,Unnamed: 0,Title,Poem,Poet,Tags
0,0,\r\r\n Objects Used to Prop...,"\r\r\nDog bone, stapler,\r\r\ncribbage board, ...",Michelle Menting,NaN
1,1,\r\r\n The New Church\r\r\n...,"\r\r\nThe old cupola glinted above the clouds,...",Lucia Cherciu,NaN
2,2,\r\r\n Look for Me\r\r\n ...,\r\r\nLook for me under the hood\r\r\nof that ...,Ted Kooser,NaN
3,3,\r\r\n Wild Life\r\r\n ...,"\r\r\nBehind the silo, the Mother Rabbit\r\r\n...",Grace Cavalieri,NaN
4,4,\r\r\n Umbrella\r\r\n ...,\r\r\nWhen I push your button\r\r\nyou fly off...,Connie Wanek,NaN


In [29]:
pdf.dropna(inplace=True)

In [30]:
pdf['single_text'] = pdf['Poem'].apply(lambda x: ' \n '.join([l.lower().strip().translate(translator) for l in x.splitlines() if len(l)>0]))
pdf.head()

,Unnamed: 0,Title,Poem,Poet,Tags,single_text
6,6,\r\r\n Invisible Fish\r\r\n...,\r\r\nInvisible fish swim this ghost ocean now...,Joy Harjo,"Living,Time & Brevity,Relationships,Family & A...",invisible fish swim this ghost ocean now descr...
7,7,\r\r\n Don’t Bother the Ear...,\r\r\nDon’t bother the earth spirit who lives ...,Joy Harjo,"Religion,The Spiritual,Mythology & Folklore,Fa...",don’t bother the earth spirit who lives here s...
9,9,"\r\r\n [""Hour in which I co...","\r\r\nHour in which I consider hydrangea, a sa...",Simone White,"Living,Parenthood,The Body,The Mind,Nature,Tre...",hour in which i consider hydrangea a salt or s...
16,16,\r\r\n scars\r\r\n ...,\r\r\nmy father’s body is a map\r\r\na record ...,Truong Tran,"The Body,Family & Ancestors",my father’s body is a map \n a record of his j...
17,17,\r\r\n what remains two\r\r...,\r\r\nit has long been forgotten this practice...,Truong Tran,"Infancy,Parenthood,The Body",it has long been forgotten this practice of th...


In [35]:
pdf.iloc[0]

Unnamed: 0                                                     6
Title          \r\r\n                    Invisible Fish\r\r\n...
Poem           \r\r\nInvisible fish swim this ghost ocean now...
Poet                                                   Joy Harjo
Tags           Living,Time & Brevity,Relationships,Family & A...
single_text    invisible fish swim this ghost ocean now descr...
Name: 6, dtype: object

In [37]:
sum_df = pd.concat([pd.DataFrame(df['single_text']), pd.DataFrame(pdf['single_text'])], ignore_index=True)
sum_df.dropna(inplace=True)


In [38]:
text_as_list = []
frequencies = {}
uncommon_words = set()
MIN_FREQUENCY = 7
MIN_SEQ = 5
BATCH_SIZE =  32
def extract_text(text):
   global text_as_list
   text_as_list += [w for w in text.split(' ') if w.strip() != '' or w == '\n']
sum_df['single_text'].apply( extract_text )
print('Total words: ', len(text_as_list))
for w in text_as_list:
   frequencies[w] = frequencies.get(w, 0) + 1
  
uncommon_words = set([key for key in frequencies.keys() if frequencies[key] < MIN_FREQUENCY])
words = sorted(set([key for key in frequencies.keys() if frequencies[key] >= MIN_FREQUENCY]))
num_words = len(words)
word_indices = dict((w, i) for i, w in enumerate(words))
indices_word = dict((i, w) for i, w in enumerate(words))
print('Words with less than {} appearances: {}'.format( MIN_FREQUENCY, len(uncommon_words)))
print('Words with more than {} appearances: {}'.format( MIN_FREQUENCY, len(words)))
valid_seqs = []
end_seq_words = []
for i in range(len(text_as_list) - MIN_SEQ ):
   end_slice = i + MIN_SEQ + 1
   if len( set(text_as_list[i:end_slice]).intersection(uncommon_words) ) == 0:
       valid_seqs.append(text_as_list[i: i + MIN_SEQ])
       end_seq_words.append(text_as_list[i + MIN_SEQ])
      
print('Valid sequences of size {}: {}'.format(MIN_SEQ, len(valid_seqs)))
X_train, X_test, y_train, y_test = train_test_split(valid_seqs, end_seq_words, test_size=0.02, random_state=42)
print(X_train[2:5])

Total words:  11989812
Words with less than 7 appearances: 600121
Words with more than 7 appearances: 79127
Valid sequences of size 5: 7442064
[['tighten', 'up', 'open', 'up', 'be'], ['chances', 'here', 'at', 'love', 'are'], ['him', 'in', 'his', 'arms', 'and']]


In [39]:
# Data generator for fit and evaluate
def generator(sentence_list, next_word_list, batch_size):
   index = 0
   while True:
       x = np.zeros((batch_size, MIN_SEQ), dtype=np.int32)
       y = np.zeros((batch_size), dtype=np.int32)
       for i in range(batch_size):
           for t, w in enumerate(sentence_list[index % len(sentence_list)]):
               x[i, t] = word_indices[w]
           y[i] = word_indices[next_word_list[index % len(sentence_list)]]
           index = index + 1
       yield x, y
# Functions from keras-team/keras/blob/master/examples/lstm_text_generation.py
def sample(preds, temperature=1.0):
   # helper function to sample an index from a probability array
   preds = np.asarray(preds).astype('float64')
   preds = np.log(preds) / temperature
   exp_preds = np.exp(preds)
   preds = exp_preds / np.sum(exp_preds)
   probas = np.random.multinomial(1, preds, 1)
   return np.argmax(probas)
def on_epoch_end(epoch, logs):
   # Function invoked at end of each epoch. Prints generated text.
   examples_file.write('\n----- Generating text after Epoch: %d\n' % epoch)
   # Randomly pick a seed sequence
   seed_index = np.random.randint(len(X_train+X_test))
   seed = (X_train+X_test)[seed_index]
 
   for diversity in [0.3, 0.4, 0.5, 0.6, 0.7]:
       sentence = seed
       examples_file.write('----- Diversity:' + str(diversity) + '\n')
       examples_file.write('----- Generating with seed:\n"' + ' '.join(sentence) + '"\n')
       examples_file.write(' '.join(sentence))
       for i in range(50):
           x_pred = np.zeros((1, MIN_SEQ))
           for t, word in enumerate(sentence):
               x_pred[0, t] = word_indices[word]
           preds = model.predict(x_pred, verbose=0)[0]
           next_index = sample(preds, diversity)
           next_word = indices_word[next_index]
 
           sentence = sentence[1:]
           sentence.append(next_word)
 
           examples_file.write(" "+next_word)
       examples_file.write('\n')
   examples_file.write('='*80 + '\n')
   examples_file.flush()

In [40]:
def get_model():
   print('Build model...')
   model = Sequential()
   model.add(Embedding(input_dim=len(words), output_dim=1024))
   model.add(Bidirectional(LSTM(128)))
   model.add(Dense(len(words)))
   model.add(Activation('softmax'))
   return model

In [ ]:
model = get_model()
model.compile(loss='sparse_categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
file_path = "./checkpoints/LSTM_LYRICS-epoch{epoch:03d}-words%d-sequence%d-minfreq%d-" \
           "loss{loss:.4f}-acc{accuracy:.4f}-val_loss{val_loss:.4f}-val_acc{val_accuracy:.4f}" % \
           (len(words), MIN_SEQ, MIN_FREQUENCY)
checkpoint = ModelCheckpoint(file_path, monitor='val_accuracy', save_best_only=True)
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)
early_stopping = EarlyStopping(monitor='val_accuracy', patience=20)
callbacks_list = [checkpoint, print_callback, early_stopping]
examples_file = open('examples.txt', "w")
model.fit(generator(X_train, y_train, BATCH_SIZE),
                   steps_per_epoch=int(len(valid_seqs)/BATCH_SIZE) + 1,
                   epochs=20,
                   callbacks=callbacks_list,
                   validation_data=generator(X_test, y_train, BATCH_SIZE),
                   validation_steps=int(len(y_train)/BATCH_SIZE) + 1)

Build model...


Epoch 1/20


  2426/232565 [..............................] - ETA: 79:35:07 - loss: 7.7230 - accuracy: 0.0701